# Generate nodes and assign their properties for the column

In [9]:
import sys,os.path
import json,yaml
import pandas as pd
import csv
import numpy as np
import math
import pprint

# sys.path.append('/Users/danieljdenman/Academics/cu/_science/models/V1\ Network\ Models\ from\ the\ Allen\ Institute/build_model/netbuilder/netbuilder')
from netbuilder.network.network import Network
import netbuilder.positions.frame_utils as fu
import netbuilder.positions.cylinder as cyl
from netbuilder.network.data_model.data_model import DataModel


# %load_ext autoreload 
# # check on every execution call
# %autoreload 2 

#%matplotlib inline

In [2]:
file_name_json = 'pop_queries.json' # get config file name from the command line argument

# load description file with the information about the populations
with open(file_name_json, 'r') as f: net_description=yaml.safe_load(f)

print 'full_path to json description:',file_name_json

#pp = pprint.PrettyPrinter(indent=2)
#pp.pprint(net_description)

full_path to json description: pop_queries.json


In [3]:
# load cell models to choose from:

cell_models_directory = '/Biophysical_network/cell_models/'

# biophysical:
bio_models_df = pd.read_csv(cell_models_directory+'/biophysical/bio_models_prop_ltd.csv',sep=' ',index_col='node_type_id');
# intfire:
bio_models_df.loc[:,'set_params_function'] = pd.Series(['biophys1']*len(bio_models_df), index=bio_models_df.index)

lif_models_df = pd.read_csv(cell_models_directory+"/intfire/lif_models_prop.csv",sep=' ',index_col='node_type_id');   
lif_models_df.loc[:,'set_params_function'] = pd.Series(['intfire1']*len(lif_models_df), index=lif_models_df.index)
lif_pop2node_type_id_df = pd.read_csv(cell_models_directory+"/intfire/lif_models_prop.csv",sep=' ',index_col='pop_name');

In [4]:
#    set somatic coordinates for the cells

network = Network() # class for building a network
network.seed(5)

pops = net_description['pops']

ncells_total = 0
cell_rarefier = 1.0
radial_range = net_description['radial_range']

for pop_name,pop_prop in pops.items():
#    print pop_prop
    depth_range = pop_prop["depth_range"]
    ncells_pop = int(round(pop_prop['ncells']*cell_rarefier)) # rarefy by a factor

    print pop_name, ': ', ncells_pop
    pop_gids = network.create_nodes(type_name=pop_name, 
                                    N=ncells_pop,
                                    instance_params = pop_prop["labels"])
    
    y_range = -np.array(depth_range)    # use negative values for y
    
    positions = cyl.generate_random_positions(ncells_pop, y_range, radial_range) 

    network.update_nodes(array_data={'x_soma': positions[:,0]}, gids=pop_gids)
    network.update_nodes(array_data={'y_soma': positions[:,1]}, gids=pop_gids)
    network.update_nodes(array_data={'z_soma': positions[:,2]}, gids=pop_gids)

    rotation_angle_yaxis = 2*math.pi*np.random.random(ncells_pop)
    network.update_nodes(array_data={'rotation_angle_yaxis': rotation_angle_yaxis }, gids=pop_gids)

    ncells_total+=ncells_pop

print 'ncells_total:',ncells_total

e5Rbp4 :  26846
e23Cux2 :  56057
i6Pvalb :  4626
e4Scnn1a :  13728
i23Pvalb :  2927
i6Htr3a :  841
i23Htr3a :  4845
e4Rorb :  11898
i5Htr3a :  508
e4other :  14644
i5Pvalb :  2876
i4Pvalb :  4461
i23Sst :  2120
i4Htr3a :  1231
i4Sst :  2384
e4Nr5a1 :  5491
i1Htr3a :  4363
i5Sst :  2538
e5noRbp4 :  6712
i6Sst :  4626
e6Ntsr1 :  57202
ncells_total: 230924


In [5]:
# Assign biophysically detailed models to cells:

# Approach: choose models for each cell in the population based on the y_soma position

length_sigma = 20 #(um) for sigma for gaussian density decay

df_list = []

model_counter = 0
print "| pop | ncells | in_query | in_range |"
for pop_name,pop_prop in pops.items():
    models_pop_df = fu.query_df(bio_models_df,pop_prop["criteria"])    # chose only models which satisfy the criteria
    models_pop_df = models_pop_df.assign(pop_name=pop_name)    

    models_range_df = fu.get_somatic_zones(models_pop_df, pop_prop["depth_range"])# return frame which includes data on allowable models
    nodes_pop = network.nodes(params={'type':pop_name}) # choose nodes from a particular population from biophysical cells
    gids_pop = network.node_ids(params={'type':pop_name})
    model_counter+=len(models_range_df)
    df_list.append(models_pop_df.loc[models_range_df.index])

   
    print pop_name, ': ', len(list(gids_pop)),' ', len(models_pop_df), ' ', len(models_range_df)
    for node in nodes_pop:  # assign model_id to individual nodes based on the value of y_soma
        y_soma = node['y_soma']
        loc_query = (models_range_df['ymin'] < y_soma) & (models_range_df['ymax'] > y_soma)     # find models which are allowed in this range

        acceptable_models_df = models_range_df[loc_query]  # get the acceptable models for this y_soma location
        if len(acceptable_models_df.index)==0:
            print y_soma    # for debugging purposes in case there are no acceptable models

        y_spec = acceptable_models_df["y_spec"].values  # y coordinate of a specimen
        prob = np.exp(-(y_soma-y_spec)**2/length_sigma**2/2)   # weight probability by a gaussian
        prob = prob/sum(prob)   # normalize probability to sum to 1.    

        node['node_type_id'] = np.random.choice(acceptable_models_df.index,p=prob)

print 'done'
print "total number of models used:", model_counter

| pop | ncells | in_query | in_range |
e5Rbp4 :  26846   10   10
e23Cux2 :  56057   2   1
i6Pvalb :  4626   6   6
e4Scnn1a :  13728   9   9
i23Pvalb :  2927   4   4
i6Htr3a :  841   4   4
i23Htr3a :  4845   8   8
e4Rorb :  11898   12   12
i5Htr3a :  508   4   4
e4other :  14644   6   6
i5Pvalb :  2876   12   12
i4Pvalb :  4461   3   3
i23Sst :  2120   4   4
i4Htr3a :  1231   1   1
i4Sst :  2384   3   3
e4Nr5a1 :  5491   10   10
i1Htr3a :  4363   2   2
i5Sst :  2538   11   9
e5noRbp4 :  6712   20   20
i6Sst :  4626   5   5
e6Ntsr1 :  57202   5   5
done
total number of models used: 138


In [6]:
#lif_models_df
bio_models_query_df = pd.concat(df_list)
len(bio_models_query_df.index)

138

In [8]:
#bio_models_query_df

In [9]:
# Assign intfire models cells:

core_radius = 400 # convert cells beyond this radius to the IntFire cells:

for pop_name,pop_prop in pops.items():
    print pop_name,
    nodes_pop = network.nodes(params={'type':pop_name}) # chose nodes from a particular population from biophysical cells
    gids_pop = network.node_ids(params={'type':pop_name})

    for node in nodes_pop:  # assign model_id to individual nodes based on the value of y_soma
        y_soma = node['y_soma']
        if node['x_soma']**2+node['z_soma']**2>core_radius**2:
            node_type_id_bio = node['node_type_id']
            node_type_id_intfire = lif_pop2node_type_id_df.loc[pop_name]["node_type_id"]
            node['node_type_id'] = node_type_id_intfire
            node['pop_name'] = 'LIF'+node["type"]
            node['level_of_detail'] = 'intfire'
        else:
            node['level_of_detail'] = 'biophysical'
            node['pop_name'] = node["type"]
print 'done'

e5Rbp4 e23Cux2 i6Pvalb e4Scnn1a i23Pvalb i6Htr3a i23Htr3a e4Rorb i5Htr3a e4other i5Pvalb i4Pvalb i23Sst i4Htr3a i4Sst e4Nr5a1 i1Htr3a i5Sst e5noRbp4 i6Sst e6Ntsr1 done


In [10]:
nodes = list(network.nodes())
nodes_df = pd.DataFrame(nodes); 
nodes_df.set_index('id',inplace=True)
nodes_df.index.name="node_id"

In [11]:
# save nodes to csv
build_directory = "build_model/V1"

cells_header = ['node_type_id','x_soma','y_soma','z_soma','rotation_angle_yaxis','pop_name','ei','location']
nodes_df.to_csv(build_directory+'/v1_nodes100.csv',columns=cells_header,sep=' ')

#bio_nodes_df.to_csv(output_dir+'/bio_nodes.csv',columns=cells_header,sep=' ')

In [24]:
# save node types to csv:

#models_df = pd.concat([bio_models_df, lif_models_df])

#models_header = ['level_of_detail','morphology_file','parameters_file','rotation_angle_zaxis','set_params_function']
#models_df.to_csv(build_directory+'/node_types.csv',columns=models_header,sep=' ',na_rep='NA')

In [12]:
header = ['level_of_detail','morphology_file','parameters_file','rotation_angle_zaxis','pop_name','set_params_function']
models_query_df = pd.concat([bio_models_query_df, lif_models_df])
models_query_df.to_csv(build_directory+'/node_types_query100.csv',columns=header,sep=' ',na_rep='NA')

In [14]:
nodes_df = pd.merge(left=bio_nodes_df,
                    right=bio_models_df, # connection labels of all cells in the network, not just those on the rank
                    how='left', 
                    left_on='model_id', 
                    right_index=True) # use 'model_id' key to merge